In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import seaborn as sns; sns.set(color_codes=True)
import numpy as np

In [2]:
species_id = pd.read_csv("~/code/skeeters/data/sample_genus_and_species.csv", header=0, index_col = 0)
species_id.rename(index={'CMS_001_RNA_A_S1':'CMS_001_RNA_A_S1_001'},inplace=True)
species_id.head()

,genus,species,corrected genus,corrected species
CMS_001_RNA_A_S1_001,Culex,erythrothorax,Culex,erythrothorax
CMS_002_10a_Rb_S119_L004,Culex,quinquefasciatus,Culex,quinquefasciatus
CMS_002_13a_Rb_S120_L004,Culex,quinquefasciatus,Culex,quinquefasciatus
CMS_002_16a_Rb_S121_L004,Aedes,aegypti,Aedes,aegypti
CMS_002_17a_Rb_S122_L004,Aedes,aegypti,Aedes,aegypti


In [3]:
dc_tsv = pd.read_csv("sketches/ska/10000000/distances.clusters.tsv", sep='\t')

In [37]:
dd_tsv = pd.read_csv("sketches/ska/10000000/distances.distances.tsv", sep='\t')
shape_dd_tsv = dd_tsv.shape

# Add prefix columns and size
dd_tsv["Sample 1 prefix"] = dd_tsv.apply(lambda row: row["Sample 1"].split("_10000000")[0], axis=1)
dd_tsv["Sample 2 prefix"] = dd_tsv.apply(lambda row: row["Sample 2"].split("_10000000")[0], axis=1)
dd_tsv["Size"] = dd_tsv.apply(lambda row: row["Sample 2"].split("_")[-1], axis=1)

dd_tsv.drop(['Sample 1', 'Sample 2'], axis=1)

# re-organize columns
dd_tsv = dd_tsv[["Sample 1 prefix", 
                 "Sample 2 prefix", 
                 "Size", "Matches", 
                 "Mismatches", 
                 "Jaccard Index",
                 "Mash-like distance",
                 "SNPs",
                 "SNP distance",
                ]]

# join on species data table
dd_tsv_w_labels = pd.merge(
    dd_tsv, 
    species_id[["genus", "species"]], 
    how='left',
    left_on="Sample 1 prefix", 
    right_index=True, 
)

# assert original data frame did not expand in row number
assert shape_dd_tsv[0] == dd_tsv_w_labels.shape[0]

In [38]:
dd_tsv_w_labels.head()

,Sample 1 prefix,Sample 2 prefix,Size,Matches,Mismatches,Jaccard Index,Mash-like distance,SNPs,SNP distance,genus,species
0,CMS_001_RNA_A_S1_001,CMS_002_10a_Rb_S119_L004_001,10000000,107591,796458,0.119010,0.049930,590,0.005484,Culex,erythrothorax
1,CMS_001_RNA_A_S1_001,CMS_002_13a_Rb_S120_L004_001,10000000,124812,898057,0.122021,0.049211,683,0.005472,Culex,erythrothorax
2,CMS_001_RNA_A_S1_001,CMS_002_16a_Rb_S121_L004_001,10000000,64767,1005138,0.060535,0.070005,591,0.009125,Culex,erythrothorax
3,CMS_001_RNA_A_S1_001,CMS_002_17a_Rb_S122_L004_001,10000000,67326,950575,0.066142,0.067318,585,0.008689,Culex,erythrothorax
4,CMS_001_RNA_A_S1_001,CMS_002_17b_Rb_S123_L004_001,10000000,68380,970402,0.065827,0.067462,593,0.008672,Culex,erythrothorax


In [39]:
# dd_tsv.Matches/(dd_tsv.Matches + dd_tsv.Mismatches)

In [40]:
# pivot table for heatmap and fill values
dd_tsv_pivot = dd_tsv.pivot(columns="Sample 1 prefix", index="Sample 2 prefix", values="Mash-like distance")
print(dd_tsv_pivot.shape)

dd_tsv_pivot = dd_tsv_pivot.loc[dd_tsv_pivot.columns, :]
print(dd_tsv_pivot.shape)

np.fill_diagonal(dd_tsv_pivot.values, 1)

dd_tsv_pivot.T.head()

dd_tsv_pivot.fillna(0) + dd_tsv_pivot.T.fillna(0) - np.diag(dd_tsv_pivot.values.diagonal())

dd_tsv_pivot.head()

(160, 160)
(160, 160)


Sample 1 prefix,CMS_001_RNA_A_S1_001,CMS_002_10a_Rb_S119_L004_001,CMS_002_13a_Rb_S120_L004_001,CMS_002_16a_Rb_S121_L004_001,CMS_002_17a_Rb_S122_L004_001,CMS_002_17b_Rb_S123_L004_001,CMS_002_17c_Rb_S124_L004_001,CMS_002_17d_Rb_S125_L004_001,CMS_002_17e_Rb_S126_L004_001,CMS_002_18a_Rb_S128_L004_001,...,CMS_055_RNA_A_S9_001,CMS_056_RNA_A_S10_001,CMS_057_RNA_A_S11_001,CMS_058_RNA_A_S9_001,CMS_059_RNA_A_S10_001,CMS_060_RNA_A_S12_001,CMS_water1_S11_001,CMS_water2_S24_001,CMS_water3_Qiagen_S26_001,CMS_water4_Zymo_S27_001
Sample 1 prefix,,,,,,,,,,,,,,,,,,,,,
CMS_001_RNA_A_S1_001,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_10a_Rb_S119_L004_001,0.049930,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_13a_Rb_S120_L004_001,0.049211,0.026395,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_16a_Rb_S121_L004_001,0.070005,0.066706,0.067499,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_17a_Rb_S122_L004_001,0.067318,0.063556,0.063863,0.027689,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
dd_tsv_pivot = dd_tsv_pivot.loc[dd_tsv_pivot.columns, :]
print(dd_tsv_pivot.shape)
dd_tsv_pivot.head()

(160, 160)


Sample 1 prefix,CMS_001_RNA_A_S1_001,CMS_002_10a_Rb_S119_L004_001,CMS_002_13a_Rb_S120_L004_001,CMS_002_16a_Rb_S121_L004_001,CMS_002_17a_Rb_S122_L004_001,CMS_002_17b_Rb_S123_L004_001,CMS_002_17c_Rb_S124_L004_001,CMS_002_17d_Rb_S125_L004_001,CMS_002_17e_Rb_S126_L004_001,CMS_002_18a_Rb_S128_L004_001,...,CMS_055_RNA_A_S9_001,CMS_056_RNA_A_S10_001,CMS_057_RNA_A_S11_001,CMS_058_RNA_A_S9_001,CMS_059_RNA_A_S10_001,CMS_060_RNA_A_S12_001,CMS_water1_S11_001,CMS_water2_S24_001,CMS_water3_Qiagen_S26_001,CMS_water4_Zymo_S27_001
Sample 1 prefix,,,,,,,,,,,,,,,,,,,,,
CMS_001_RNA_A_S1_001,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_10a_Rb_S119_L004_001,0.049930,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_13a_Rb_S120_L004_001,0.049211,0.026395,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_16a_Rb_S121_L004_001,0.070005,0.066706,0.067499,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_17a_Rb_S122_L004_001,0.067318,0.063556,0.063863,0.027689,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
np.fill_diagonal(dd_tsv_pivot.values, 1)
dd_tsv_pivot.head()

Sample 1 prefix,CMS_001_RNA_A_S1_001,CMS_002_10a_Rb_S119_L004_001,CMS_002_13a_Rb_S120_L004_001,CMS_002_16a_Rb_S121_L004_001,CMS_002_17a_Rb_S122_L004_001,CMS_002_17b_Rb_S123_L004_001,CMS_002_17c_Rb_S124_L004_001,CMS_002_17d_Rb_S125_L004_001,CMS_002_17e_Rb_S126_L004_001,CMS_002_18a_Rb_S128_L004_001,...,CMS_055_RNA_A_S9_001,CMS_056_RNA_A_S10_001,CMS_057_RNA_A_S11_001,CMS_058_RNA_A_S9_001,CMS_059_RNA_A_S10_001,CMS_060_RNA_A_S12_001,CMS_water1_S11_001,CMS_water2_S24_001,CMS_water3_Qiagen_S26_001,CMS_water4_Zymo_S27_001
Sample 1 prefix,,,,,,,,,,,,,,,,,,,,,
CMS_001_RNA_A_S1_001,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_10a_Rb_S119_L004_001,0.049930,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_13a_Rb_S120_L004_001,0.049211,0.026395,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_16a_Rb_S121_L004_001,0.070005,0.066706,0.067499,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CMS_002_17a_Rb_S122_L004_001,0.067318,0.063556,0.063863,0.027689,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
dd_tsv_pivot.T.head()

Sample 1 prefix,CMS_001_RNA_A_S1_001,CMS_002_10a_Rb_S119_L004_001,CMS_002_13a_Rb_S120_L004_001,CMS_002_16a_Rb_S121_L004_001,CMS_002_17a_Rb_S122_L004_001,CMS_002_17b_Rb_S123_L004_001,CMS_002_17c_Rb_S124_L004_001,CMS_002_17d_Rb_S125_L004_001,CMS_002_17e_Rb_S126_L004_001,CMS_002_18a_Rb_S128_L004_001,...,CMS_055_RNA_A_S9_001,CMS_056_RNA_A_S10_001,CMS_057_RNA_A_S11_001,CMS_058_RNA_A_S9_001,CMS_059_RNA_A_S10_001,CMS_060_RNA_A_S12_001,CMS_water1_S11_001,CMS_water2_S24_001,CMS_water3_Qiagen_S26_001,CMS_water4_Zymo_S27_001
Sample 1 prefix,,,,,,,,,,,,,,,,,,,,,
CMS_001_RNA_A_S1_001,1.0,0.04993,0.049211,0.070005,0.067318,0.067462,0.068545,0.068126,0.066185,0.068100,...,0.028684,0.028783,0.025650,0.018694,0.031471,0.052075,0.164545,0.113581,0.039834,0.113309
CMS_002_10a_Rb_S119_L004_001,NaN,1.00000,0.026395,0.066706,0.063556,0.064974,0.068096,0.065291,0.062453,0.063623,...,0.043487,0.043796,0.044650,0.048381,0.035761,0.062671,0.140467,0.100328,0.045961,0.111583
CMS_002_13a_Rb_S120_L004_001,NaN,NaN,1.000000,0.067499,0.063863,0.064900,0.066195,0.065417,0.062911,0.067050,...,0.046685,0.047403,0.048390,0.048015,0.034248,0.065055,0.148104,0.094093,0.046100,0.101497
CMS_002_16a_Rb_S121_L004_001,NaN,NaN,NaN,1.000000,0.027689,0.028000,0.031360,0.027436,0.030702,0.031406,...,0.063525,0.066162,0.066782,0.068673,0.069380,0.064701,0.149992,0.113852,0.067518,0.120642
CMS_002_17a_Rb_S122_L004_001,NaN,NaN,NaN,NaN,1.000000,0.024006,0.026997,0.024175,0.026575,0.028464,...,0.061299,0.062832,0.063333,0.066141,0.066653,0.062020,0.143412,0.103246,0.063995,0.110311


In [49]:
dd_tsv_pivot.fillna(0) + dd_tsv_pivot.T.fillna(0) - np.diag(dd_tsv_pivot.values.diagonal())

Sample 1 prefix,CMS_001_RNA_A_S1_001,CMS_002_10a_Rb_S119_L004_001,CMS_002_13a_Rb_S120_L004_001,CMS_002_16a_Rb_S121_L004_001,CMS_002_17a_Rb_S122_L004_001,CMS_002_17b_Rb_S123_L004_001,CMS_002_17c_Rb_S124_L004_001,CMS_002_17d_Rb_S125_L004_001,CMS_002_17e_Rb_S126_L004_001,CMS_002_18a_Rb_S128_L004_001,...,CMS_055_RNA_A_S9_001,CMS_056_RNA_A_S10_001,CMS_057_RNA_A_S11_001,CMS_058_RNA_A_S9_001,CMS_059_RNA_A_S10_001,CMS_060_RNA_A_S12_001,CMS_water1_S11_001,CMS_water2_S24_001,CMS_water3_Qiagen_S26_001,CMS_water4_Zymo_S27_001
Sample 1 prefix,,,,,,,,,,,,,,,,,,,,,
CMS_001_RNA_A_S1_001,1.000000,0.049930,0.049211,0.070005,0.067318,0.067462,0.068545,0.068126,0.066185,0.068100,...,0.028684,0.028783,0.025650,0.018694,0.031471,0.052075,0.164545,0.113581,0.039834,0.113309
CMS_002_10a_Rb_S119_L004_001,0.049930,1.000000,0.026395,0.066706,0.063556,0.064974,0.068096,0.065291,0.062453,0.063623,...,0.043487,0.043796,0.044650,0.048381,0.035761,0.062671,0.140467,0.100328,0.045961,0.111583
CMS_002_13a_Rb_S120_L004_001,0.049211,0.026395,1.000000,0.067499,0.063863,0.064900,0.066195,0.065417,0.062911,0.067050,...,0.046685,0.047403,0.048390,0.048015,0.034248,0.065055,0.148104,0.094093,0.046100,0.101497
CMS_002_16a_Rb_S121_L004_001,0.070005,0.066706,0.067499,1.000000,0.027689,0.028000,0.031360,0.027436,0.030702,0.031406,...,0.063525,0.066162,0.066782,0.068673,0.069380,0.064701,0.149992,0.113852,0.067518,0.120642
CMS_002_17a_Rb_S122_L004_001,0.067318,0.063556,0.063863,0.027689,1.000000,0.024006,0.026997,0.024175,0.026575,0.028464,...,0.061299,0.062832,0.063333,0.066141,0.066653,0.062020,0.143412,0.103246,0.063995,0.110311
CMS_002_17b_Rb_S123_L004_001,0.067462,0.064974,0.064900,0.028000,0.024006,1.000000,0.027090,0.024193,0.027364,0.029352,...,0.061696,0.063539,0.064224,0.066658,0.067070,0.062515,0.145362,0.106172,0.064520,0.112404
CMS_002_17c_Rb_S124_L004_001,0.068545,0.068096,0.066195,0.031360,0.026997,0.027090,1.000000,0.027876,0.029809,0.030132,...,0.064533,0.066572,0.066844,0.067905,0.068166,0.064917,0.151087,0.099997,0.064792,0.105438
CMS_002_17d_Rb_S125_L004_001,0.068126,0.065291,0.065417,0.027436,0.024175,0.024193,0.027876,1.000000,0.026628,0.030057,...,0.062525,0.061651,0.063010,0.067130,0.068969,0.063960,0.143231,0.104535,0.064158,0.111332
CMS_002_17e_Rb_S126_L004_001,0.066185,0.062453,0.062911,0.030702,0.026575,0.027364,0.029809,0.026628,1.000000,0.029352,...,0.059226,0.061170,0.061566,0.065595,0.065714,0.061048,0.140370,0.099289,0.063179,0.106643


In [ ]:
dd_tsv_pivot.values

In [ ]:
t = distances_distances_tsv[[""]]
#sns.pairplot()

In [ ]:
iris = sns.load_dataset("iris")
iris.head()

In [ ]:
from khtools import knn, sourmash_utils

In [ ]:
pd.__file__